In [1]:
!pip install sqlalchemy==1.3.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 77.1 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp37-cp37m-linux_x86_64.whl size=1159121 sha256=3d1660516259b5ccaeada3345065a790c2d332f7b5b0f640b32bb69946d69d24
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/03/71/13/010faf12246f72dc76b4150e6e599d13a85b4435e06fb9e51f
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.24
    Uninstalling SQLAlchemy-1.3.24:
      Successfully uninstalled SQLAlchemy-1.3.24


In [2]:
%load_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
!pip install -q pandas==1.1.5

In [5]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [6]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  both result in 0.1234 being formatted as 0.12.


In [7]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

In [8]:
# Execute the SQL query to display unique launch sites
result = %sql SELECT DISTINCT "Launch_Site" FROM SPACEXTABLE
result

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [9]:
# Execute the SQL query to display 5 records where launch sites begin with 'KSC'
result_ksc = %sql SELECT * FROM SPACEXTABLE WHERE "Launch_Site" LIKE 'KSC%' LIMIT 5
result_ksc

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2017-02-19,14:39:00,F9 FT B1031.1,KSC LC-39A,SpaceX CRS-10,2490,LEO (ISS),NASA (CRS),Success,Success (ground pad)
2017-03-16,6:00:00,F9 FT B1030,KSC LC-39A,EchoStar 23,5600,GTO,EchoStar,Success,No attempt
2017-03-30,22:27:00,F9 FT B1021.2,KSC LC-39A,SES-10,5300,GTO,SES,Success,Success (drone ship)
2017-05-01,11:15:00,F9 FT B1032.1,KSC LC-39A,NROL-76,5300,LEO,NRO,Success,Success (ground pad)
2017-05-15,23:21:00,F9 FT B1034,KSC LC-39A,Inmarsat-5 F4,6070,GTO,Inmarsat,Success,No attempt


In [10]:
# Execute the SQL query to calculate the total payload mass for NASA (CRS) missions
result_nasa_crs = %sql SELECT SUM("PAYLOAD_MASS__KG_") AS Total_Payload_Mass FROM SPACEXTABLE WHERE "Mission_Outcome" LIKE '%NASA (CRS)%'
result_nasa_crs

 * sqlite:///my_data1.db
Done.


Total_Payload_Mass
None


In [11]:
# Execute the SQL query to calculate the average payload mass for booster version F9 v1.1
result_avg_payload = %sql SELECT AVG("PAYLOAD_MASS__KG_") AS Average_Payload_Mass FROM SPACEXTABLE WHERE "Booster_Version" = 'F9 v1.1'
result_avg_payload

 * sqlite:///my_data1.db
Done.


Average_Payload_Mass
2928.4


In [12]:
# Execute the SQL query to find the earliest date of a successful landing on a drone ship
result_successful_landing = %sql SELECT MIN(Date) AS First_Successful_Landing_Date FROM SPACEXTABLE WHERE "Landing_Outcome" = 'Success (drone ship)'
result_successful_landing

 * sqlite:///my_data1.db
Done.


First_Successful_Landing_Date
2016-04-08


In [13]:
# Execute the SQL query to list booster names with successful ground pad landings and payload mass between 4000 and 6000 kg
result_boosters = %sql SELECT "Booster_Version" FROM SPACEXTABLE WHERE "Landing_Outcome" = 'Success (ground pad)' AND "PAYLOAD_MASS__KG_" > 4000 AND "PAYLOAD_MASS__KG_" < 6000
result_boosters

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1032.1
F9 B4 B1040.1
F9 B4 B1043.1


In [14]:
# Execute the SQL query to count the number of successful and failed mission outcomes
result_mission_outcomes = %sql SELECT "Mission_Outcome", COUNT(*) AS Count FROM SPACEXTABLE GROUP BY "Mission_Outcome"
result_mission_outcomes

 * sqlite:///my_data1.db
Done.


Mission_Outcome,Count
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


In [15]:
# Execute the SQL query to find the booster versions which have carried the maximum payload mass using a subquery
result_max_payload_booster = %sql SELECT "Booster_Version" FROM SPACEXTABLE WHERE "PAYLOAD_MASS__KG_" = (SELECT MAX("PAYLOAD_MASS__KG_") FROM SPACEXTABLE)
result_max_payload_booster

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [16]:
# Execute the SQL query to list records for successful ground pad landings, booster versions, and launch sites for the months in 2017
result_2017_records = %sql SELECT substr(Date, 6, 2) AS Month, "Landing_Outcome", "Booster_Version", "Launch_Site" FROM SPACEXTABLE WHERE substr(Date, 0, 5) = '2017' AND "Landing_Outcome" = 'Success (ground pad)'
result_2017_records

 * sqlite:///my_data1.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
02,Success (ground pad),F9 FT B1031.1,KSC LC-39A
05,Success (ground pad),F9 FT B1032.1,KSC LC-39A
06,Success (ground pad),F9 FT B1035.1,KSC LC-39A
08,Success (ground pad),F9 B4 B1039.1,KSC LC-39A
09,Success (ground pad),F9 B4 B1040.1,KSC LC-39A
12,Success (ground pad),F9 FT B1035.2,CCAFS SLC-40


In [17]:
# Execute the SQL query to rank the count of landing outcomes between the specified dates in descending order
result_landing_outcomes_ranked = %sql SELECT "Landing_Outcome", COUNT(*) AS Count FROM SPACEXTABLE WHERE Date BETWEEN '2010-06-04' AND '2017-03-20' GROUP BY "Landing_Outcome" ORDER BY Count DESC
result_landing_outcomes_ranked

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
